In [ ]:
# Load dotenv
import os
from dotenv import load_dotenv

load_dotenv(override=True)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("Groq key loaded:", GROQ_API_KEY is not None)
print("Tavily key loaded:", TAVILY_API_KEY is not None)
print("OpenAI key loaded:", OPENAI_API_KEY is not None)
print("Groq key:", GROQ_API_KEY)
print("Tavily key:", TAVILY_API_KEY)
print("OpenAI key:", OPENAI_API_KEY)

In [2]:
# Connect Groq LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=GROQ_API_KEY,
    model_name="llama-3.1-8b-instant",
    temperature=0.2,
)

response = llm.invoke("Explain what RAG is in simple terms.")
print(response.content)

RAG is an acronym that stands for "Red, Amber, Green." It's a simple way to categorize or rate things, often used in business, project management, and quality control.

Here's how it works:

- **Red (R)**: This indicates a problem or a critical issue that needs immediate attention. It's like a warning sign that says, "Something's wrong, and we need to fix it now."

- **Amber (A)**: This means there's a potential issue or a warning sign that needs to be monitored. It's like a caution sign that says, "Be careful, something might go wrong if we don't take action."

- **Green (G)**: This indicates that everything is okay, and there are no major issues. It's like a thumbs-up sign that says, "We're good to go, and everything is working as it should."

RAG is often used to track progress, identify potential problems, and make informed decisions. It's a simple and effective way to communicate and prioritize tasks.


In [4]:
# Load and Split Doc
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("crew_data/doc.pdf")
documents = loader.load()

print("Total pages loaded:", len(documents))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(documents)

print("Number of chunks:", len(docs))

Total pages loaded: 6
Number of chunks: 32


In [5]:
# Filter content
filtered_docs = [
    doc for doc in docs
    if "PMID" not in doc.page_content
    and "doi:" not in doc.page_content
]

print("After filtering:", len(filtered_docs))

After filtering: 23


In [ ]:
# Check OpenAI key
print(os.environ.get("OPENAI_API_KEY"))

In [7]:
# Create embeddings (OpenAI - have to pay)
from langchain_openai import OpenAIEmbeddings

embeddings_charge = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

print("Embedding model ready")

Embedding model ready


In [8]:
# Free Embeddings (Local - No API needed)
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Free embedding model ready")

d:\PPunch\Data Science Internship Program\project03-agentic-RAG-app\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 159.41it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Free embedding model ready


In [9]:
# Build FAISS vectorstore
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(filtered_docs, embeddings)
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6}
)

print("Vectorstore created successfully")

Vectorstore created successfully


In [10]:
# Create LLM with Groq
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=GROQ_API_KEY,
    model_name="llama-3.1-8b-instant",
    temperature=0.2,
)

print("Groq LLM ready")

Groq LLM ready


In [11]:
# Create RetrievalQA Chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

print("QA chain ready")

QA chain ready


In [12]:
# Test asking a question from PDF
query = "What is RAG?"

response = qa_chain.invoke({"query": query})

print(response["result"])

I don't know what RAG stands for in the given context.


In [13]:
query = "What is the role of generative AI in healthcare?"

response = qa_chain.invoke({"query": query})

print(response["result"])

According to the provided context, the role of generative AI in healthcare is multifaceted. It is being explored for its potential to automate clinical workflow, particularly in clinical summarization. Generative AI models, such as Large Language Models (LLMs), can process large amounts of information for summarization and analysis, which can help alleviate the burden of information overload on physicians.

Some specific applications of generative AI in healthcare mentioned in the context include:

1. Automating clinical workflow: Generative AI can help automate tasks such as disease detection and report generation.
2. Clinical summarization: Generative AI models can be used to summarize lengthy and complex clinical reports, such as radiology reports, to provide essential information for diagnosing and monitoring diseases.
3. Improving data analysis: Generative AI can help analyze large amounts of data, including unstructured clinical information, to provide insights that can inform cl

In [14]:
# Test Retrieval
query = "What is the role of generative AI in healthcare?"
results = retriever.get_relevant_documents(query)

print("Top result:\n")
print(results[0].page_content)

Top result:

effectiveness in healthcare delivery while others fear issues in data privacy, ethical considerations, and more. This
debate is further complicated with the advent of generative AI and its impact on workflows across the industry. However,
it is no surprise that AI that automates clinical workflow is an exciting frontier. It is an undeniable truth that generative
AI is finding its foothold cautiously in the hands of physicians – this review article will explore the current state of
clinical summarization in healthcare, and how AI pushes its frontiers to previously unexplored heights.arXiv:2407.16905v1  [cs.CY]  24 Jun 2024


d:\PPunch\Data Science Internship Program\project03-agentic-RAG-app\rag_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [15]:
# Create Tavily
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

response = tavily_client.search(
    query="Latest advancements in generative AI in healthcare",
    search_depth="basic",
    max_results=3
)

response

{'query': 'Latest advancements in generative AI in healthcare',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.xr.health/eu/us/blog/generative-ai-in-healthcare/',
   'title': '10 Breakthrough Innovations - Generative AI in Healthcare',
   'content': 'Generative AI is transforming healthcare, paving the way for new diagnostics, treatment, and patient engagement opportunities.',
   'score': 0.73891455,
   'raw_content': None},
  {'url': 'https://www.mckinsey.com/industries/healthcare/our-insights/generative-ai-in-healthcare-current-trends-and-future-outlook',
   'title': 'Generative AI in healthcare: Current trends and future outlook',
   'content': 'Generative AI in healthcare can enhance operations and engage stakeholders, with many advancing to full-scale implementation.',
   'score': 0.7192461,
   'raw_content': None},
  {'url': 'https://healthtechmagazine.net/article/2025/01/overview-2025-ai-trends-healthcare',
   'title': 'An Overvie

In [30]:
# Create tools for agent
from crewai_tools import tool

@tool
def tavily_search_tool(query: str) -> str:
    """
    Search the web using Tavily and return relevant results.
    """
    results = tavily_client.search(
        query=query,
        search_depth="basic",
        max_results=2
    )

    # return content only
    texts = [r["content"][:500] for r in results["results"]]
    return "\n\n".join(texts)

@tool
def local_rag_search(query: str) -> str:
    """
    Search the local PDF knowledge base for relevant information.
    """

    # get top 2 only
    filtered_docs = retriever.invoke(query)[:2]
    return "\n\n".join([filtered_docs.page_content for filtered_docs in filtered_docs])

print("Tools ready")

Tools ready


In [34]:
# Create Agents
from crewai import Agent

researcher = Agent(
    role="Researcher",
    goal="Find accurate and detailed information using both local knowledge base and web search.",
    backstory="An expert researcher skilled in retrieving relevant information from documents and the web.",
    tools=[local_rag_search, tavily_search_tool],
    llm=llm,
    max_iter=3,         # limit working
    verbose=True
)

writer = Agent(
    role="Writer",
    goal="Write a comprehensive and well-structured report based on the research findings.",
    backstory="A professional writer skilled in summarizing complex information clearly.",
    llm=llm,
    max_iter=3,         # limit working
    verbose=True
)

critic = Agent(
    role="Critic",
    goal="Review the report for accuracy, coherence, and completeness.",
    backstory="A critical reviewer ensuring factual correctness and clarity.",
    llm=llm,
    max_iter=3,         # limit working
    verbose=True
)

In [35]:
# Create Tasks
from crewai import Task

research_task = Task(
    description="Research the role of generative AI in healthcare.",
    expected_output="A detailed summary of findings from local knowledge and web search.",
    agent=researcher
)

write_task = Task(
    description="Write a detailed and well-structured report based on the research findings.",
    expected_output="A comprehensive written report.",
    agent=writer
)

review_task = Task(
    description="Review the report and suggest improvements.",
    expected_output="A revised version of the report with improvements.",
    agent=critic
)

In [36]:
# Create crew and run
from crewai import Crew

crew = Crew(
    agents=[researcher, writer],    #, critic],
    tasks=[research_task, write_task],  #, review_task],
    process="sequential",
    verbose=True
)

result = crew.kickoff()
print(result)

2026-02-19 12:03:47,409 - 31436 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: Research the role of generative AI in healthcare.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by gathering information on the role of generative AI in healthcare. I should search the local knowledge base to see if there are any relevant documents or studies.

Action: local_rag_search
Action Input: {"query": "generative AI in healthcare"} 

effectiveness in healthcare delivery while others fear issues in data privacy, ethical considerations, and more. This
debate is further complicated with the advent of generative AI and its impact on workflows across the industry. However,
it is no surprise that AI that automates clinical workflow is an exciting frontier. It is an undeniable truth that generative
AI is finding its foothold cautiously in the hands of physicians – this review article will explore the current state of
clinical summarization in healthcare, and how AI pushes its frontiers to previous

In [4]:
import sys
print(sys.executable)

d:\PPunch\Data Science Internship Program\project03-agentic-RAG-app\rag_env\Scripts\python.exe
